In [20]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)
 
# 입력X는 28x28 크기의 이미지들이며 그레이스케일 포맷이므로 채널은 1개
# 출력 Y는 원 핫 인코딩된 텐서들이며 [0,0,1,0,0,0,0,0,0,0]는 2를 뜻함.
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)
 
# 합성곱 계층
# 커널(필터)은 3x3 크기이며 입력과 똑같은 1채널임.
# 커널의 개수는 하이퍼 파라미터이므로 적당히 32개로 한다.
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
# 입력X와 커널W1의 합성곱 연산을 정의한다.
# 스트라이드는 1이다. ([batch, height, width, channel] 형식의 텐서)
# padding을 'SAME'으로 하면 출력의 크기가 줄어들지 않는다.
L1 = tf.nn.conv2d(X, W1, strides=[1, 1, 1, 1], padding='SAME')
L1 = tf.nn.relu(L1)
 
# 풀링 계층
# 2x2 영역에서 가장 큰 값 하나만을 취한다.
# 스트라이드는 2다.
# 때문에 출력의 크기는 반으로 줄어든다. (28x28 -> 14x14)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
    padding='SAME')
 
# 합성곱 계층
# 크기는 이전과 똑같지만 이전의 필터 개수가 32개였으므로 3번째 Shape이 32다.
# 이번에는 커널의 개수를 64개로 한다.
# 나머지는 이전과 동일하다.
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
 
# 풀링 계층
# 이전과 동일하다.
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
    padding='SAME')
 
# 완전 연결 신경망
# 입력 7*7*64개, 출력 256개의 히든 레이어
W3 = tf.Variable(tf.random_normal([7 * 7 * 64, 256], stddev=0.01))
# 이전 신경망의 결과를 1차원 텐서로 Reshape한다.
# [-1, 7 * 7 * 64]는 행의 개수는 자동으로 계산하고
# 열의 개수는 이전 계층의 출력인 (7x7x64)의 크기와 같다.
L3 = tf.reshape(L2, [-1, 7 * 7 * 64])
L3 = tf.nn.relu(tf.matmul(L3, W3))
# 드롭아웃
L3 = tf.nn.dropout(L3, keep_prob)
 
# 출력 계층
# 이전 계층의 출력 개수인 256개의 입력과 숫자의 개수인 10개의 출력을 가짐.
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)
 
# 오차 함수는 교차 엔트로피 오차 함수를 사용.
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    labels=Y, logits=model
))
 
# 최적화는 AdamOptimizer를 사용한다.
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
 
    batch_size = 100
    total_batch = int(mnist.train.num_examples / batch_size)
 
    for epoch in range(15):
        total_cost = 0
 
        for i in range(total_batch):
            # 학습할 데이터를 미니배치로 얻어오는데
            # 입력은 nx28x28x1로 Reshape 한다.
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            batch_xs = batch_xs.reshape(-1, 28, 28, 1)
 
            _, cost_val = sess.run([optimizer, cost],
                feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
 
            total_cost += cost_val
 
        print('Epoch:', '%04d' % (epoch + 1),
            'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
 
    print('완료!')
 
    is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    print('정확도: %.2f' % sess.run(accuracy * 100,
        feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1),
            Y: mnist.test.labels, keep_prob: 1}))

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 Avg. cost = 0.362
Epoch: 0002 Avg. cost = 0.108
Epoch: 0003 Avg. cost = 0.077
Epoch: 0004 Avg. cost = 0.063
Epoch: 0005 Avg. cost = 0.049
Epoch: 0006 Avg. cost = 0.042
Epoch: 0007 Avg. cost = 0.036
Epoch: 0008 Avg. cost = 0.033
Epoch: 0009 Avg. cost = 0.029
Epoch: 0010 Avg. cost = 0.025
Epoch: 0011 Avg. cost = 0.023
Epoch: 0012 Avg. cost = 0.020
Epoch: 0013 Avg. cost = 0.018
Epoch: 0014 Avg. cost = 0.016
Epoch: 0015 Avg. cost = 0.015
완료!
정확도: 98.99
